## Import the libraries

In [1]:
#importing the libraries
import tensorflow as tf  
from tensorflow import keras
import numpy as np
import pandas as pd

## Load the data and data pre-processing

In [2]:
#load the datasets
baseline_df = pd.read_excel('extracted_features_baseline.xlsx')
toolwear_df = pd.read_excel('extracted_features_toolwear.xlsx')

In [3]:
#concantanate the datasets
combined_df = pd.concat([baseline_df, toolwear_df], axis=0)

In [4]:
features = combined_df.values

In [5]:
print(features.shape[1])

66


In [8]:
#train test split
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(features, test_size=0.2, random_state=50)

In [9]:
#data scalling
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Construct and train the Autoencoder

In [10]:
#specify the number of condensed features. This will be the number of neurons in the hidden layer
condensed_f = 10

In [11]:
#constructing the model

#input layer which number of neurons equals the number of original features
l_in = keras.Input(features.shape[1])

#hidden layer which condenses the feature into the specified number of condensed features
l_condensed = keras.layers.Dense(condensed_f)(l_in)

#output layer which is the same as the input
l_out = keras.layers.Dense(features.shape[1])(l_condensed)

In [12]:
#defining the autoencode
autoencoder = keras.Model(l_in, l_out)

In [13]:
autoencoder.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 66)]              0         
                                                                 
 dense (Dense)               (None, 10)                670       
                                                                 
 dense_1 (Dense)             (None, 66)                726       
                                                                 
Total params: 1396 (5.45 KB)
Trainable params: 1396 (5.45 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:
#compile the model
autoencoder.compile(optimizer='adam', loss='mse')
#train the model
autoencoder.fit(X_train, X_train, epochs = 50, batch_size = 8, validation_split = 0.1)


Epoch 1/50

76/76 [==============================] - 1s 5ms/step - loss: 1.0560 - val_loss: 0.7014
Epoch 2/50
76/76 [==============================] - 0s 2ms/step - loss: 0.6715 - val_loss: 0.4666
Epoch 3/50
76/76 [==============================] - 0s 2ms/step - loss: 0.4783 - val_loss: 0.3769
Epoch 4/50
76/76 [==============================] - 0s 2ms/step - loss: 0.3933 - val_loss: 0.3327
Epoch 5/50
76/76 [==============================] - 0s 2ms/step - loss: 0.3482 - val_loss: 0.3010
Epoch 6/50
76/76 [==============================] - 0s 2ms/step - loss: 0.3174 - val_loss: 0.2754
Epoch 7/50
76/76 [==============================] - 0s 2ms/step - loss: 0.2925 - val_loss: 0.2522
Epoch 8/50
76/76 [==============================] - 0s 3ms/step - loss: 0.2703 - val_loss: 0.2314
Epoch 9/50
76/76 [==============================] - 0s 2ms/step - loss: 0.2501 - val_loss: 0.2126
Epoch 10/50
76/76 [==============================] - 0s 2ms/step - loss: 0.2324 - val_loss: 0.1981
Epoch 11/50
76/76

## Evaluate Autoencoder with training dataset

In [15]:
from sklearn.metrics import mean_absolute_error

#Predict with the model and record the MSE between the output and input features
pred = autoencoder.predict(X_test)
print(mean_absolute_error(X_test,pred))

6/6 [==============================] - 0s 2ms/step
0.2751215868966251
